In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

Data cleaning

In [2]:
loans_df = pd.read_csv("LoansData.csv")

In [3]:
loans_df.columns = loans_df.columns.str.replace(".","")
loans_df.InterestRate=loans_df.InterestRate.str.replace("%","").astype(float)
loans_df.DebtToIncomeRatio=loans_df.DebtToIncomeRatio.str.replace("%","").astype(float)

loans_df.EmploymentLength = loans_df.EmploymentLength.str.replace("< ","").str.replace(" years","").str.replace(" year","").str.replace("+","").astype(float)

C:\Users\sainiaal\AppData\Local\Temp/ipykernel_2728/1262380196.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  loans_df.columns = loans_df.columns.str.replace(".","")
C:\Users\sainiaal\AppData\Local\Temp/ipykernel_2728/1262380196.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  loans_df.EmploymentLength = loans_df.EmploymentLength.str.replace("< ","").str.replace(" years","").str.replace(" year","").str.replace("+","").astype(float)


In [4]:
for col in loans_df.select_dtypes(object).columns:
    loans_df[col]=loans_df[col].fillna(loans_df[col].mode()[0])
for col in loans_df.select_dtypes("number").columns:
    loans_df[col]=loans_df[col].fillna(loans_df[col].mean())

In [5]:
loans_df.isna().sum()

AmountRequested              0
AmountFundedByInvestors      0
InterestRate                 0
LoanLength                   0
LoanPurpose                  0
DebtToIncomeRatio            0
State                        0
HomeOwnership                0
MonthlyIncome                0
FICORange                    0
OpenCREDITLines              0
RevolvingCREDITBalance       0
InquiriesintheLast6Months    0
EmploymentLength             0
dtype: int64

In [6]:
loans_df.describe(percentiles=[0.01,0.99]).T

,count,mean,std,min,1%,50%,99%,max
AmountRequested,2500.0,12405.462185,7801.372299,1000.00,1500.0000,10000.000000,35000.0000,35000.00
AmountFundedByInvestors,2500.0,12002.374186,7745.217219,-0.01,999.1728,10000.000000,35000.0000,35000.00
InterestRate,2500.0,13.065388,4.175529,5.42,5.9880,13.110000,22.9533,24.89
DebtToIncomeRatio,2500.0,15.380420,7.503551,0.00,0.7190,15.320000,33.2300,34.91
MonthlyIncome,2500.0,5688.931321,3962.325165,588.50,1416.5366,5000.000000,18750.0000,102750.00
OpenCREDITLines,2500.0,10.072887,4.504710,2.00,3.0000,9.000000,23.0000,38.00
RevolvingCREDITBalance,2500.0,15223.184622,18270.038964,0.00,0.0000,10977.500000,92407.6600,270800.00
InquiriesintheLast6Months,2500.0,0.906688,1.230410,0.00,0.0000,0.906688,5.0000,9.00
EmploymentLength,2500.0,5.491952,3.388514,1.00,1.0000,5.000000,10.0000,10.00


In [7]:
for col in ["MonthlyIncome","OpenCREDITLines","RevolvingCREDITBalance"]:
    upp_cut = loans_df[col].quantile(0.99)
    low_cut = loans_df[col].quantile(0.01)
    loans_df[col] = loans_df[col].clip(low_cut,upp_cut)

In [8]:
loans_df.describe().T

,count,mean,std,min,25%,50%,75%,max
AmountRequested,2500.0,12405.462185,7801.372299,1000.0000,6000.00,10000.000000,17000.0000,35000.00
AmountFundedByInvestors,2500.0,12002.374186,7745.217219,-0.0100,6000.00,10000.000000,16000.0000,35000.00
InterestRate,2500.0,13.065388,4.175529,5.4200,10.16,13.110000,15.8000,24.89
DebtToIncomeRatio,2500.0,15.380420,7.503551,0.0000,9.75,15.320000,20.6725,34.91
MonthlyIncome,2500.0,5608.582367,3083.750057,1416.5366,3500.00,5000.000000,6800.0000,18750.00
OpenCREDITLines,2500.0,10.054087,4.377253,3.0000,7.00,9.000000,13.0000,23.00
RevolvingCREDITBalance,2500.0,14769.469222,14756.904656,0.0000,5589.25,10977.500000,18843.7500,92407.66
InquiriesintheLast6Months,2500.0,0.906688,1.230410,0.0000,0.00,0.906688,1.0000,9.00
EmploymentLength,2500.0,5.491952,3.388514,1.0000,2.00,5.000000,10.0000,10.00


In [9]:
loans_df.duplicated().sum()

0

In [10]:
loans_df["FICORange"]=(loans_df.FICORange.str[4:7].astype(int) + loans_df.FICORange.str[:3].astype(int))//2

In [11]:
loans_df["LoanLength"]=loans_df.LoanLength.map({"36 months": 36,"60 months":60})

<h3>PROBLEM 1

In [12]:
#a

#H0: independent variables
#Ha : As Amount increases interest rate decreases
#confidence : 95%
#alpha :5%
#test = pearsons coefficient test 
stats.pearsonr(loans_df.AmountRequested,loans_df.InterestRate)

(0.33244640596656827, 1.4148683739824417e-65)

the r-squared suggests there is a weak positive relationship between Interest Rate and Amount Requested
and the p value is very small giving enough evidence to reject the null hypothesis
hence Interest rate and Amount are dependent on each other

In [13]:
#b

#H0: independent Variables
#Ha : As LoanLength increases interest rate increases
#confidence : 95%
#alpha :5%
#test = pearsons coefficient test
stats.pearsonr(loans_df.LoanLength,loans_df.InterestRate)

(0.42417192416620586, 9.54959606435366e-110)

the r-squared suggests there is a moderate positive relationship between Interest Rate and Loan Length and the p value is very small giving enough evidence to reject the null hypothesis 

In [14]:
#c

#H0: Intrest rate is same for different purpose of loans
#Ha : Intrest rate varies for different purpose of loans
#confidence : 95%
#alpha :5%
#decision rule : +1.96>stat>-1.96
#test = pearsons coefficient test
seq = []
for val in loans_df.LoanPurpose.unique():
    seq.append(loans_df[loans_df.LoanPurpose == val].InterestRate)
#print(len(seq))
stats.f_oneway(seq[0],seq[1],seq[2],seq[3],seq[4],seq[5],seq[6],seq[7],seq[8],seq[9],seq[10],seq[11],seq[12],seq[13])

F_onewayResult(statistic=7.482638784097435, pvalue=1.1599665535989115e-14)

The p value is very small giving enough evidence to reject the null hypothesis
which means that interest rate varies for different loan purposes

In [15]:
#d

#H0: independent
#Ha : People with owning home will have high FICO scores(dependent)
#confidence : 95%
#alpha :5%
#decision rule : +1.96>stat>-1.96
#test = pearsons coefficient test
df1 = np.where(loans_df.HomeOwnership == "OWN",1,0)
df2 = loans_df.FICORange
stats.pearsonr(df1,df2)

(0.0056728067974554115, 0.7767929930372584)

the r-squared suggests there is pretty weak positive relationship between Fico scores and Homeownership and the p value is very high giving enough evidence to accept the null hypothesis hence both are independent variables

<h3>PROBLEM 2


In [16]:
pricing_df = pd.read_csv("Price_Quotes.csv")

In [17]:
#H0: avg of MARY == avg of BARRY
#Ha : avg of MARY != avg of BARRY
#confidence : 95%
#alpha :0.05
#decision rule : +1.96>stat>-1.96
#test = T-test

stats.ttest_ind(pricing_df.Mary_Price,pricing_df.Barry_Price)

Ttest_indResult(statistic=-1.4147436739281787, pvalue=0.17114226132118285)

The p value is greater than the alpha region and there is not enough evidence to reject the null hypothesis hence we accept the null hypothesis

<h3>PROBLEM 3

In [18]:
facility_df = pd.read_csv("Treatment_Facility.csv")
facility_df.rename(columns={"VAR4":"TRFF","VAR5":"CI"},inplace=True)
prior_df = facility_df[facility_df.Reengineer == "Prior"]
post_df = facility_df[facility_df.Reengineer == "Post"]

In [44]:
#H0: no changes
#Ha :there were changes after the Reengineering
#confidence : 90%
#alpha :0.10
#test = t test

stats.ttest_ind(prior_df.Employee_Turnover,post_df.Employee_Turnover)

Ttest_indResult(statistic=-1.770306704753604, pvalue=0.09361109345535304)

the p value is smaller than the alpha giving enough evidence to reject the null hypothesis
there has been an increase in people quitting after the reengineering as the stat value is in negative

In [20]:
#H0: no changes
#Ha :there were changes after the Reengineering
#confidence : 90%
#alpha :0.10
#test = t test

stats.ttest_ind(prior_df.TRFF,post_df.TRFF)

Ttest_indResult(statistic=2.7833984488804506, pvalue=0.012265082138734354)

the p value is smaller than the alpha giving enough evidence to reject the null hypothesis
there has been a decrease in residents who were temporarily removed from the facility
after the reengineering as the stat value is in very high.

In [36]:
#H0: prior !< post
#Ha : prior<post
#confidence : 90%
#alpha :0.10
#test = t -test

stats.ttest_ind(prior_df.CI,post_df.CI,alternative="less")

Ttest_indResult(statistic=1.6279144253528646, pvalue=0.9395400540505793)

the p value is greater than the alpha giving enough evidence to accept the null hypothesis,
hence we can say that CI has decreased after the reengineering



<h3>PROBLEM 4

In [3]:
p_asses_df = pd.read_csv("Priority_Assessment.csv")

In [4]:
high_prio = p_asses_df[p_asses_df.Priority == "High"].Days
med_prio = p_asses_df[p_asses_df.Priority == "Medium"].Days
low_prio = p_asses_df[p_asses_df.Priority == "Low"].Days

In [8]:
#H0 : high priority tasks takes less days than medium priority tasks
#Ha : high priority tasks takes more days than medium priority tasks
#confidence : 95%
#alpha :0.05
#test = t test

stats.ttest_ind(high_prio,med_prio,alternative="greater")

Ttest_indResult(statistic=0.925239411198133, pvalue=0.17761544090583975)

since the p value is higher than the alpha region we can say that we have enough evidence to accept the null hypothesis 
hence we can say that high priority tasks were done faster than the medium priority tasks

In [48]:
#H0 : medium priority tasks takes less days than low priority tasks
#Ha : medium priority tasks takes more days than low priority tasks
#confidence : 95%
#alpha :0.05
#test = t test

stats.ttest_ind(med_prio,low_prio,alternative="greater")

Ttest_indResult(statistic=-2.3690822147998514, pvalue=0.9907814954922975)

since the p value is higher than the alpha region we can say that we have enough evidence to accept the null hypothesis 
hence we can say that medium priority tasks were done faster than the low priority tasks

<h3>PROBLEM 5

In [2]:
Films = pd.read_csv("Films.csv")

In [3]:
Films['Hear_About'] = Films['Hear_About'].fillna(Films['Hear_About'].mode()[0])
Films['Sinage'] = Films['Sinage'].fillna(Films['Sinage'].mode()[0])
Films['Parking'] = Films['Parking'].fillna(Films['Parking'].mode()[0])
Films['Clean'] = Films['Clean'].fillna(Films['Clean'].mode()[0])
Films['Overall'] = Films['Overall'].fillna(Films['Overall'].mode()[0])
Films['Income'] = Films['Income'].fillna(Films['Income'].mean())
Films['Age'] = Films['Age'].fillna(Films['Age'].mode()[0])

In [4]:
Films["Gender"] = np.where(Films.Gender.str.startswith("Fe"),2,1)
Films["Marital_Status"] = np.where(Films.Marital_Status.str.startswith("Marr"),1,2)
Films["Hear_About"] = Films.Hear_About.fillna(0)
Films["Income"] = Films.Income.fillna(round(Films.Income.mean()))

In [5]:
Films.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _rowstate_      330 non-null    int64  
 1   Movie           330 non-null    object 
 2   Gender          330 non-null    int32  
 3   Marital_Status  330 non-null    int32  
 4   Sinage          330 non-null    float64
 5   Parking         330 non-null    float64
 6   Clean           330 non-null    float64
 7   Overall         330 non-null    float64
 8   Age             330 non-null    float64
 9   Income          330 non-null    float64
 10  Hear_About      330 non-null    object 
dtypes: float64(6), int32(2), int64(1), object(2)
memory usage: 25.9+ KB


In [6]:
#1 What isthe overall level of customer satisfaction?
Films.Overall.mean()


1.621212121212121

In [7]:
#2 What factors are linked to satisfaction?

Films.select_dtypes("number").corr()["Overall"].sort_values(ascending=False)[1:4]


Parking    0.514429
Sinage     0.382103
Clean      0.350822
Name: Overall, dtype: float64

In [9]:
obs_freq1 = pd.crosstab( Films.Sinage, Films.Overall )
obs_freq2 = pd.crosstab( Films.Clean, Films.Overall )
obs_freq3 = pd.crosstab( Films.Parking, Films.Overall )
chi1,p1,dot1,ex1 = stats.chi2_contingency(obs_freq1)
chi2,p2,dot2,ex2 = stats.chi2_contingency(obs_freq2)
chi3,p3,dot3,ex3 = stats.chi2_contingency(obs_freq3)

print(chi1,chi2,chi3)
print(p1,p2,p3)

125.61859671027881 127.71945555999517 230.61691663391676
4.53298970003405e-19 1.7772535915434025e-19 4.783162283608494e-40


Parking has the most positive relationship with Overall satisfaction followed by sinage and Cleanliness


In [10]:
#3 What is the demographic profile of Film on the Rocks patrons?

Films.Gender.value_counts()/len(Films)*100
age = Films.groupby("Age").agg("count")
gender = Films.groupby("Gender").agg("count")
marry = Films.groupby("Marital_Status").agg("count")
hear = Films.groupby("Hear_About").agg("count")
c_index = ["Age","Age","Age","Age","Gender","Gender","Marital_Status","Marital_Status"]
index = ["1-12","13-30","31-60","60+","Male","Female","Married","Single"]
df = pd.concat([age,gender,marry]).reset_index()
df["index"] = index
df["Columns"] = c_index
df = df.set_index(["Columns","index"])
df = df.iloc[:,0:1]
df.columns = ["Frequency"] 
df["%"] = round(df.Frequency / 330 * 100,2)
df

Frequency      %
Columns        index                    
Age            1-12            26   7.88
               13-30          177  53.64
               31-60          117  35.45
               60+             10   3.03
Gender         Male           311  94.24
               Female          19   5.76
Marital_Status Married         14   4.24
               Single         316  95.76

In [36]:

#4 In what media outlet(s) should the film series be advertised?

# H0: There is no relationship between overall ratings and hear_abouts
# Ha : There is a relationship 
# confidence = 95% , alpha = 0.05
# Chi squared test

obs1 = pd.crosstab(Films.Overall,Films.Hear_About)
chi_stats ,pvalue,dof,expected = stats.chi2_contingency(obs1)

if pvalue > 0.05:
    print("Failed to reject the null hypothesis")
else:
    print("Rejected the null hypothesis")

Failed to reject the null hypothesis


In [39]:
fd = Films.Hear_About.value_counts()

In [71]:
values = {1:0,2:0,3:0,4:0,5:0}
for val in fd.index :
    if val.startswith("1") or val.endswith("1"):
        values[1]=fd[val]+values[1]
    if val.startswith("2") or val.endswith("2"):
        values[2]=fd[val]+values[2]
    if val.startswith("3") or val.endswith("3"):
        values[3]=fd[val]+values[3]
    if val.startswith("4") or val.endswith("4"):
        values[4]=fd[val]+values[4]
    if val.startswith("5") or val.endswith("5"):
        values[5]=fd[val]+values[5]
        
df = pd.DataFrame(values.values(),index=values.keys(),columns=["Frequency"]).sort_values(by="Frequency",ascending=False)
df.index = np.where(df.index ==  1 ,"television",np.where(df.index == 2 ,"newspaper",np.where(df.index == 3, "radio",np.where(df.index == 4 ,"website", "word of mouth"))))
df

,Frequency
word of mouth,239
website,45
television,23
radio,17
newspaper,14


the best working advertisement method is the word of mouth with the highest number of people 
where website advertising comes second